In [1]:
subdir_name = 'pdfs/' # Include / as last char
core_name = 'Town Revenues' # Use to Infer name.pdf output and also .csv filenames
file_loc_in_bucket = 'jpg_organized/' + subdir_name + core_name + '.pdf'

In [2]:
import boto3
import textractor
from textractprettyprinter.t_pretty_print import Pretty_Print_Table_Format, Textract_Pretty_Print, get_string

In [3]:
# Config
REGION = aws_region = boto3.session.Session().region_name # 'us-west-2'
BUCKET = bucket_name = 'andrew-ocr'

In [4]:
s3 = boto3.client('s3', REGION)
textract = boto3.client('textract', aws_region) 

In [5]:
"""For reference"""
# def analyze_pdf_document_with_a2i(document_name, bucket):
#     response = textract.start_document_analysis(
#         DocumentLocation={'S3Object': {'Bucket': bucket, 'Name': document_name}},
#         FeatureTypes=["TABLES", "FORMS"], 
# #         HumanLoopConfig=humanLoopConfig
#     )
#     return response

'For reference'

In [6]:
analyzePDF = textract.start_document_analysis(
    DocumentLocation={'S3Object': {'Bucket': bucket_name, 'Name': file_loc_in_bucket}},
    FeatureTypes=["TABLES"] # Careful! TABLES, FORMS
)

analyzePDF_results= textract.get_document_analysis(
    JobId=analyzePDF['JobId']
)

In [7]:
def GetResults(jobId):
    maxResults = 1000
    paginationToken = None
    finished = False
    pages= []
    while finished == False:
        response = None
#         if self.processType == ProcessType.ANALYSIS:
        if paginationToken == None:
#             response = self.textract.get_document_analysis(JobId=jobId, MaxResults=maxResults)
            response = textract.get_document_analysis(JobId=jobId)
        else:
#             response = self.textract.get_document_analysis(JobId=jobId, MaxResults=maxResults, NextToken=paginationToken)
            response = textract.get_document_analysis(JobId=jobId, NextToken=paginationToken)
#         if self.processType == ProcessType.DETECTION:
#            if paginationToken == None:
#                response = self.textract.get_document_text_detection(JobId=jobId, MaxResults=maxResults)
#            else:
#                response = self.textract.get_document_text_detection(JobId=jobId,MaxResults=maxResults, NextToken=paginationToken)
        blocks = response['Blocks']
        print('Detected Document Text')
        print('Pages: {}'.format(response['DocumentMetadata']['Pages']))
        # Display block information
#         for block in blocks:
#            self.DisplayBlockInfo(block)
#            print()
#            print()
        if 'NextToken' in response:
           paginationToken = response['NextToken']
        else:
            finished = True
#             return response
    # Put response on pages list
        pages.append(response)
        print('Document Detected.')

        if 'NextToken' in response:
            paginationToken = response['NextToken']
        else:
            finished = True

    # convert pages as JSON pattern
    pages = json.dumps(pages)
    return pages



# Note: 
You may have to wait for results to finsh processing (in the cloud) before executing the rest of the cells. 

In [12]:
JOBID = analyzePDF['JobId']
analyzePDF_results_all = GetResults(JOBID)
outputInJsonText= str(analyzePDF_results_all)

output_filename = core_name + '.json'
s3.put_object(Body=outputInJsonText, Bucket=BUCKET, Key=output_filename)

Detected Document Text
Pages: 15
Document Detected.
Detected Document Text
Pages: 15
Document Detected.
Detected Document Text
Pages: 15
Document Detected.
Detected Document Text
Pages: 15
Document Detected.
Detected Document Text
Pages: 15
Document Detected.
Detected Document Text
Pages: 15
Document Detected.
Detected Document Text
Pages: 15
Document Detected.
Detected Document Text
Pages: 15
Document Detected.
Detected Document Text
Pages: 15
Document Detected.
Detected Document Text
Pages: 15
Document Detected.
Detected Document Text
Pages: 15
Document Detected.


{'ResponseMetadata': {'RequestId': '7GDGRGQ8W947Z6ZP',
  'HostId': 'bVCCr6SGec6fjWhF44Q6JIcIanqfBnzSgAz/SVopqVQnQZk8VOVEwQqmT+j76go6ld9dVyG11sE=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'bVCCr6SGec6fjWhF44Q6JIcIanqfBnzSgAz/SVopqVQnQZk8VOVEwQqmT+j76go6ld9dVyG11sE=',
   'x-amz-request-id': '7GDGRGQ8W947Z6ZP',
   'date': 'Mon, 17 Jan 2022 18:36:14 GMT',
   'x-amz-version-id': 'DWpP9LymmJu0UqX99w4mLrPFHMNyi9Fu',
   'etag': '"d23fd59a87eed7013e0a91c237e6bf0d"',
   'server': 'AmazonS3',
   'content-length': '0'},
  'RetryAttempts': 0},
 'ETag': '"d23fd59a87eed7013e0a91c237e6bf0d"',
 'VersionId': 'DWpP9LymmJu0UqX99w4mLrPFHMNyi9Fu'}

In [13]:
output = s3.get_object(Bucket=BUCKET, Key=output_filename)

streaming_obj = output['Body']
string_data = streaming_obj.read().decode("utf-8")

json_data= json.loads(string_data)

In [14]:
textract_json = json_data
textractor_csv = get_string(textract_json=textract_json,
               table_format=Pretty_Print_Table_Format.csv,
               output_type=[Textract_Pretty_Print.TABLES, Textract_Pretty_Print.FORMS])

In [15]:
prefix= 'csvs/'
output_file = prefix + core_name +'.csv'

# replace content
with open(output_file, "wt") as fout:
    fout.write(textractor_csv)